In [1]:
%%javascript
IPython.notebook.clear_all_output();

<IPython.core.display.Javascript object>

# Load the abstract file

In [30]:
import numpy as np
import pandas as pd

file = 'modified_abstract.json'
data = pd.read_json(file, lines=True)

Full File Shape

In [31]:
data.shape

(17597123, 3)

Delete Empty Data

In [32]:
data = data.dropna(subset=['abstract','docid'])

Cleaned File Shape

In [33]:
data.shape

(9354165, 3)

In [34]:
data.head()

,abstract,docid,title
0,A Brazilian family with hereditary gingival fi...,7800376,Hereditary gingival fibromatosis. Report of an...
1,Necrosis that occurs in a salivary gland neopl...,7800377,Necrosis in benign salivary gland neoplasms. N...
2,Seventy-nine patients with oral leukoplakia th...,7800378,A clinical trial of antioxidant supplements in...
3,Proliferative verrucous leukoplakia is a slow-...,7800379,Proliferative verrucous leukoplakia. Four case...
4,We report the case of a patient in whom oral s...,7800380,A case of histiocytic medullary reticulosis co...


# Load the gene file

In [8]:
gile = 'gene.csv'
gene = pd.read_csv(gile)
gene.head()

,Unnamed: 0,TAX_ID,GENE_ID,SYMBOL,SYNONYMES
0,0,9606,28,ABO,A3GALNT|A3GALT1|GTB|NAGAT
1,1,9606,29,ABR,MDB
2,2,9606,30,ACAA1,ACAA|PTHIO|THIO
3,3,9606,31,ACACA,ACAC|ACACAD|ACC|ACC1|ACCA
4,4,9606,32,ACACB,ACC2|ACCB|HACC275


In [9]:
gene_dic = {}

index = 0
gene_ids = gene['GENE_ID']
symbols = gene['SYMBOL']
synonymes = gene['SYNONYMES']
while True:
    if gene_ids[index]:
        try:
            if symbols[index]:
                gene_dic[symbols[index].lower()] =  gene_ids[index]
            if synonymes[index]:
                synonyme = synonymes[index].split('|')
            for syn in synonyme:
                gene_dic[syn.lower()] =  gene_ids[index]
            index += 1
        except:
            break

# Clean Abstract Rows Without Genetic Info

In [11]:
abstracts = data['abstract']

In [13]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

keys = gene_dic.keys()

index = 0
delete_index = []
for abstr in abstracts:
    word_tokens = word_tokenize(abstr)
    detector = 0
    for token in word_tokens:
        if token in keys:
            detector = 1
            break
    if detector == 0:
        delete_index.append(index)
    index += 1


[nltk_data] Downloading package punkt to /home/weidatan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [62]:
len(delete_index)

5843

In [29]:
file = open('delete_index.txt','a')
for info in delete_index:
    file.write(str(info))
    file.write('\n')
file.close()

In [39]:
i = 0
for delete in delete_index:
    new_data = data.drop(data.index[[delete-i]])
    i += 1

File Shape with Genetic Info

In [40]:
data.shape

(9354165, 3)

In [41]:
data.head()

,abstract,docid,title
0,A Brazilian family with hereditary gingival fi...,7800376,Hereditary gingival fibromatosis. Report of an...
1,Necrosis that occurs in a salivary gland neopl...,7800377,Necrosis in benign salivary gland neoplasms. N...
2,Seventy-nine patients with oral leukoplakia th...,7800378,A clinical trial of antioxidant supplements in...
3,Proliferative verrucous leukoplakia is a slow-...,7800379,Proliferative verrucous leukoplakia. Four case...
4,We report the case of a patient in whom oral s...,7800380,A case of histiocytic medullary reticulosis co...


In [42]:
data.to_pickle('cleaned_abstract.pickle')
#new_data = pd.read_pickle('cleaned_abstract.pickle')
#new_data.shape
#new_data.head()

# Tokenization

In [43]:
abstracts = data['abstract']

In [46]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from gensim.parsing.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

stem_abstracts = []
for abstr in abstracts:
    word_tokens = word_tokenize(abstr)
    stem_abstr = []
    for token in word_tokens:
        #stem_token = PorterStemmer().stem(token)
        #if re.search('[a-zA-Z0-9]',stem_token):
            #stem_abstr.append(stem_token)
        if re.search('[a-zA-Z0-9]',token):
            stem_abstr.append(token)
    #print(abstr)
    #print(stem_abstr)
    #break
    stem_abstracts.append(stem_abstr)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/weidatan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/weidatan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
from gensim.models import Word2Vec

model = Word2Vec(stem_abstracts, size=100, window=5, min_count=1, workers=4)

model.save("w2v.model")

End